In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, optimizers, Sequential
import glob
import numpy as np
import random

In [ ]:
train_paths, label_paths = [],[]
with open("dataset_sidewalk/train.txt", "r") as f:
    for line in f.readlines():
        train_name, label_name = line.strip().split(";")
        train_paths.append("dataset_sidewalk/jpg/"+train_name)
        label_paths.append("dataset_sidewalk/png/"+label_name)
print( len(train_paths) , len(label_paths) ) 
print( train_paths[0:2], label_paths[0:2])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train_paths, label_paths, test_size = 0.25, random_state = 888888)
print(len(x_train), len(x_valid), len(y_train), len(y_valid))
print(x_train[0:2], x_valid[0:2], y_train[0:2], y_valid[0:2])

In [ ]:
def data_processing(img, label):
    img = tf.io.read_file(img)
    img = tf.image.decode_jpeg( img , channels = 3)
    img = tf.cast(img , dtype = "float32")
    label = tf.io.read_file(label)
    label = tf.image.decode_jpeg( label , channels = 1)
    label = tf.cast(label , dtype = "int32")
    return img/255., label

In [ ]:
img, label = data_processing(x_train[20], y_train[20])
print(img.shape, label.shape)
print(np.max(img), np.min(img))
print(np.max(label), np.min(label))

In [ ]:
train_db = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_db = train_db.shuffle(10).map(data_processing).batch(1)

valid_db = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
valid_db = valid_db.shuffle(10).map(data_processing).batch(1)

sample1 = next(iter(train_db))
print('sample1:', sample1[0].shape, sample1[1].shape)
sample2 = next(iter(valid_db))
print('sample2:', sample2[0].shape, sample2[1].shape)

# Model

In [ ]:
baseModel = tf.keras.applications.VGG16(weights="imagenet",include_top=False,input_shape = (720, 1280, 3),pooling=None)
baseModel.trainable = False
contractingNet = tf.keras.models.Model(inputs=baseModel.input, outputs=baseModel.get_layer('block4_pool').output)

f1 = contractingNet.get_layer("block1_conv2").output
f2 = contractingNet.get_layer("block2_conv2").output
f3 = contractingNet.get_layer("block3_conv3").output
f4 = contractingNet.get_layer("block4_conv3").output

a = layers.Conv2D(1024, kernel_size=[3, 3], padding="same", activation="relu")(contractingNet.output)
a = layers.Conv2D(512, kernel_size=[1, 1], padding="same", activation="relu")(a)
a = layers.Conv2D(1024, kernel_size=[3, 3], padding="same", activation="relu")(a)

a = layers.UpSampling2D(size = (2, 2))(a)
a = layers.concatenate([a,f4], axis = -1)
a = layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation="relu")(a)
a = layers.Conv2D(256, kernel_size=[1, 1], padding="same", activation="relu")(a)
a = layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation="relu")(a)

a = layers.UpSampling2D(size = (2, 2))(a)
a = layers.concatenate([a,f3], axis = -1)
a = layers.Conv2D(256, kernel_size=[3, 3], padding="same", activation="relu")(a)
a = layers.Conv2D(128, kernel_size=[1, 1], padding="same", activation="relu")(a)
a = layers.Conv2D(256, kernel_size=[3, 3], padding="same", activation="relu")(a)

a = layers.UpSampling2D(size = (2, 2))(a)
a = layers.concatenate([a,f2], axis = -1)
a = layers.Conv2D(128, kernel_size=[3, 3], padding="same", activation="relu")(a)
a = layers.Conv2D(64, kernel_size=[1, 1], padding="same", activation="relu")(a)
a = layers.Conv2D(128, kernel_size=[3, 3], padding="same", activation="relu")(a)

a = layers.UpSampling2D(size = (2, 2))(a)
a = layers.concatenate([a,f1], axis = -1)
a = layers.Conv2D(64, kernel_size=[3, 3], padding="same", activation="relu")(a)
a = layers.Conv2D(32, kernel_size=[1, 1], padding="same", activation="relu")(a)
a = layers.Conv2D(64, kernel_size=[3, 3], padding="same", activation="relu")(a)

a = layers.Conv2D(8, kernel_size=[3, 3], padding="same", activation="relu")(a)
outputs = layers.Conv2D(1, kernel_size=[1, 1], padding="same", activation="sigmoid")(a)

model = tf.keras.Model(inputs = baseModel.input, outputs = outputs )
model.summary()

# Training

In [ ]:
optimizer = optimizers.Adam(lr=1e-4)
model.compile(loss = "binary_crossentropy", optimizer = optimizer,metrics = ["acc"])

model_path = "models/model-sidewalk-{epoch:04d}-{val_acc:.4f}.h5"
cp = keras.callbacks.ModelCheckpoint(model_path, monitor = "val_acc", save_best_only = True, mode="max")
callbacks = [cp]

In [ ]:
history = model.fit(train_db, epochs = 20, validation_data = valid_db, callbacks = callbacks)

# Load model to evaluate

In [ ]:
del model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, optimizers, Sequential
import glob
import numpy as np
import random

In [ ]:
model = tf.keras.models.load_model('models/model-sidewalk-0016-0.9945.h5')

In [ ]:
def get_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg( img , channels = 3)
    img = tf.cast(img , dtype = "float32")
    img = tf.image.resize(img, [720, 1280])
    img = tf.expand_dims(img, axis = 0)
    return img/255.

In [ ]:
train_paths, label_paths = [],[]
with open("dataset_sidewalk/train.txt", "r") as f:
    for line in f.readlines():
        train_name, label_name = line.strip().split(";")
        train_paths.append("dataset_sidewalk/jpg/"+train_name)
        label_paths.append("dataset_sidewalk/png/"+label_name)
print( len(train_paths) , len(label_paths) ) 
print( train_paths[0:2], label_paths[0:2])

In [ ]:
import cv2
startindex = 30
watch_paths = x_valid[startindex:startindex+6]
for i, path in enumerate(watch_paths):
    img = get_img(path)
    orgimg = tf.squeeze(img, axis = 0)
    pred = model.predict(img)
    pred = tf.squeeze(pred, axis = 0)
    pred_pad = tf.pad(pred , [[0,0],[0,0],[1,1]])
    addimg = orgimg*0.9 + pred_pad*0.3
    cv2.imshow('addimg'+str(i), addimg.numpy())
    
cv2.waitKey(0)
cv2.destroyAllWindows()